<a href="https://colab.research.google.com/github/woo2201/sampleCases/blob/reservior/SpaceTitanic_toyplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
cd /content/drive/MyDrive/dataset/SpaceTitanic/

/content/drive/MyDrive/dataset/SpaceTitanic


In [137]:
pip install sklearn.data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [138]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [126]:
path = "/content/drive/MyDrive/dataset/SpaceTitanic/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[-1] for var in filenames]

# dynamically allocate df names.
df_list = []
for name in zip(filenames, data):
    loc = list(name)[0]
    val = list(name)[1]
    if not val.startswith('HomeCredit'):
        df_list.append(val)
        globals()[val] = pd.read_csv(loc) # encoding = 'ISO-8859-1'

In [5]:
train.sample(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
2225,2384_05,Earth,True,G/383/S,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,NaN,0.0,Ronia Pollarkeeks,False
8094,8646_01,Europa,True,C/325/S,TRAPPIST-1e,48.0,False,0.0,0.0,0.0,0.0,0.0,Neutrak Vertfull,True
7356,7871_01,Earth,True,NaN,TRAPPIST-1e,18.0,False,0.0,0.0,0.0,0.0,0.0,Alendy Stannondez,True


In [6]:
test.sample(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
1151,2425_01,Earth,True,G/396/P,TRAPPIST-1e,0.0,False,0.0,0.0,NaN,0.0,0.0,Raulia Tranklinay
138,0299_01,Earth,False,F/58/S,55 Cancri e,20.0,False,376.0,0.0,242.0,1.0,0.0,Dellia Hickett
3635,7925_01,Earth,False,F/1518/S,TRAPPIST-1e,24.0,False,0.0,56.0,975.0,0.0,0.0,Elanie Thorney


In [127]:
(train.shape[0] == train.PassengerId.nunique(), test.shape[0] == test.PassengerId.nunique())

(True, True)

In [128]:
pd.crosstab(train.VIP, train.Destination)

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
VIP,,,
False,1692,756,5667
True,65,18,114


In [159]:
def feature_engineering(df, dummyVariables):

    if isinstance(df, pd.DataFrame) and isinstance(dummyVariables, list):

        df = (
                df
                .assign(Psng_grp_1 = lambda x : x.PassengerId.str.split("_").apply(lambda x: x[0]))
                .assign(Psng_grp_2 = lambda x : x.PassengerId.str.split("_").apply(lambda x: x[1]))
        )

        CameAlone = [i for i, j in zip(df.Psng_grp_1.value_counts().index, df.Psng_grp_1.value_counts().values) if j == 1]

        df = (  df
                .assign(totalSpent = lambda x : x.RoomService	+ x.FoodCourt + x.ShoppingMall + x.Spa	+ x.VRDeck)
                .assign(Accompanied = lambda x : x.Psng_grp_1.apply(lambda x: 'Alone' if x in CameAlone else 'Accompanied'))
                .assign(Cabin_Type = lambda x : x.Cabin.str.split('/').apply(lambda x: x[2] if isinstance(x, list) else np.NaN))
                .assign(underAge = lambda x: x.Age.apply(lambda x: 1 if x <= 19 else 0))
        )
    
        df = pd.get_dummies(df, columns = dummyVariables)

    return df

In [168]:
train_df = feature_engineering(train, dummyVariables = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination'])
test_df = feature_engineering(test, dummyVariables = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination'])

In [169]:
train_keys = train_df[['PassengerId', 'Psng_grp_1', 'Psng_grp_2']]
test_keys = test_df[['PassengerId', 'Psng_grp_1', 'Psng_grp_2']]

target = train_df.Transported

In [170]:
cols_to_drop = ['Cabin', 'Name', 'PassengerId', 'Psng_grp_1', 'Psng_grp_2']
for df in [train_df, test_df]:
    df.drop(cols_to_drop, axis = 1, inplace = True)
    df['Cabin_Type'] = df.Cabin_Type.apply(lambda x: 1 if x == 'P' else 0)
    df['Accompanied'] = df.Accompanied.apply(lambda x: 1 if x == 'Accompanied' else 0)

In [178]:
train_df.drop(['Transported'], axis = 1, inplace = True)

XGBOOST only takes numerical values and is not able to measure the distance between any order-based values. 

In [179]:
xgb_model = XGBRegressor(objective = "reg:squarederror",
                         learning_Rate = 0.05,
                         max_depth = 3,
                         n_estimators = 200)
xgb_model.fit(train_df, target, verbose = False)

XGBRegressor(learning_Rate=0.05, n_estimators=200, objective='reg:squarederror')

In [180]:
# make predictions
predictions =  xgb_model.predict(test_df)

In [193]:
predictions_binary = [True if x >= 0.5 else False for x in predictions]

In [194]:
from collections import Counter
Counter(predictions_binary)

Counter({False: 1990, True: 2287})

In [196]:
sample_submission.Transported = predictions_binary

In [198]:
sample_submission.to_csv("rslt_1.csv", index = False)